# OpenAI tests for lambda - azure function converter

## Testing Chain of Thought Prompting (CoT)

### Install the necessary libraries

In [1]:
%pip install langchain
%pip install openai
%pip install chromadb
%pip install tiktoken
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Setup

Import the libraries and environment variables to gain access to the `Open API Key`

In [54]:
import os
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key=os.environ['OPENAI_API_KEY']
base_url=os.environ['OPENAI_BASE_URL']
api_version=os.environ['OPENAI_API_VERSION']

print(base_url)

https://devsquad-eastus-2.openai.azure.com/


Let's create the llm client, we can use a chat llm from langchain

In [55]:
llm = AzureChatOpenAI(
    api_key=api_key,
    azure_endpoint=base_url, 
    api_version=api_version,
    model="gpt-4",
    temperature=0
)

#### Create embeddings of your documents to get ready for semantic search

We are going to get our embeddings engine ready. This will be the engine that will turn out documents into vector embeddings so we can easily do semantic search on them.

In [56]:
embeddings = AzureOpenAIEmbeddings(
    api_key=api_key,
    azure_endpoint=base_url, 
    api_version=api_version,
    azure_deployment="text-embedding-ada-002"
)

In [57]:
text = "this is a test document"
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])
doc_result[0][:3]

[-0.012222584727053142, 0.007210398239221619, -0.014818063280923785]

### Create the example set
To get started, create a list of few-shot examples. Each example should be a dictionary with the keys being the input variables and the values being the values for those input variables.

**Note:** An improvement could be to add metadata to the examples to help recognize the most similar ones

Let's start by getting a list of paths with the examples

In [58]:
def list_files_in_folders(path):
    return os.listdir(path)

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

In [59]:
basePath = "../go-examples/examples/gin/"
pathList = list_files_in_folders(basePath)
print(pathList)

['basic-conversion-1', 'basic-conversion-2', 'basic-conversion-3', 'basic-conversion-4', 'basic-conversion-5', 's3-conversion-1']


In [60]:
examples = []

for path in pathList:
    input = read_file(basePath + path + "/input/main.go")
    output = read_file(basePath + path + "/output/main.go")
    exampleName = path
    example = {
        "input": input, 
        "output": output,
        # "metadata": {"name": exampleName, "language": "go"}
    }
    examples.append(example)

Checkout one example and see if it's good

In [61]:
print(examples[0])

{'input': 'package main\n\nimport (\n\t"context"\n\t"fmt"\n\n\t"github.com/aws/aws-lambda-go/lambda"\n)\n\ntype MyEvent struct {\n\tName string `json:"name"`\n}\n\ntype MyResponse struct {\n\tMessage string `json:"message"`\n}\n\nfunc HandleRequest(ctx context.Context, event *MyEvent) (*MyResponse, error) {\n\tif event == nil {\n\t\treturn nil, fmt.Errorf("received nil event")\n\t}\n\tmessage := fmt.Sprintf("Hello %s!", event.Name)\n\treturn &MyResponse{Message: message}, nil\n}\n\nfunc main() {\n\tlambda.Start(HandleRequest)\n}\n', 'output': 'package main\n\nimport (\n\t"fmt"\n\t"log"\n\t"net/http"\n\t"os"\n\n\t"github.com/gin-gonic/gin"\n)\n\nconst (\n\tEnvVarAzureFunctionPort = "FUNCTIONS_PORT"\n)\n\ntype MyEvent struct {\n\tName string `json:"name"`\n}\n\ntype MyResponse struct {\n\tMessage string `json:"message"`\n}\n\nfunc HandleRequest(ctx *gin.Context)  {\n\tif ctx.Request.Body == nil {\n\t\terrorMsg := "received nil event"\n\t\tctx.JSON(http.StatusBadRequest, gin.H{"error": er

### Test input

Let's define the target input that we'll use to select the examples and finally to run our query

In [62]:
input = read_file("../go-examplesexamples/test-inputs/basic-input-1/main.go")

### Using Advanced Chain-of-Thought (CoT) Prompt

Add a system input that explains the steps the LLM needs to follow

In [63]:
system_input = '''You are an azure assistant that converts aws lambda code into azure functions. 
    You do it by using the following steps:
    
    1. Determine if the code corresponds to a lambda function, by checking its imports
    2. If it isn't a lambda function, return it as is
    3. If it is a lambda function then convert the file to an azure function
   
   Return the resultant code, and only code do not add any explanation.
   
   If you don't know the answer return the original code with the following comment: 
   `// Could not convert the code, please check the class`, followed by a commented explanation of what happened.
   '''

### Using an example selector

We will reuse the example set and the formatter from the previous section. However, instead of feeding the examples directly into the `FewShotPromptTemplate` object, we will feed them into an `ExampleSelector` object.

In this tutorial, we will use the `SemanticSimilarityExampleSelector` class. This class selects few-shot examples based on their similarity to the input. It uses an embedding model to compute the similarity between the input and the few-shot examples, as well as a vector store to perform the nearest neighbor search.

In [64]:
# metadatas = [example["metadata"] for example in examples]
to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [65]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": input})

[{'input': 'package main\n\nimport (\n\t"context"\n\t"fmt"\n\t"github.com/aws/aws-lambda-go/lambda"\n)\n\ntype MyEvent struct {\n\tName string `json:"name"`\n}\n\ntype MyResponse struct {\n\tMessage string `json:"message"`\n}\n\nfunc HandleRequest(ctx context.Context, event *MyEvent) (*MyResponse, error) {\n\tif event == nil {\n\t\treturn nil, fmt.Errorf("received nil event")\n\t}\n\tmessage := fmt.Sprintf("Hello %s!", event.Name)\n\treturn &MyResponse{Message: message}, nil\n}\n\nfunc main() {\n\tlambda.Start(HandleRequest)\n}\n',
  'output': 'package main\n\nimport (\n\t"fmt"\n\t"log"\n\t"net/http"\n\t"os"\n\n\t"github.com/gin-gonic/gin"\n)\n\nconst (\n\tEnvVarAzureFunctionPort = "FUNCTIONS_PORT"\n)\n\ntype MyEvent struct {\n\tName string `json:"name"`\n}\n\ntype MyResponse struct {\n\tMessage string `json:"message"`\n}\n\nfunc HandleRequest(ctx *gin.Context)  {\n\tif ctx.Request.Body == nil {\n\t\terrorMsg := "received nil event"\n\t\tctx.JSON(http.StatusBadRequest, gin.H{"error": e

### Create a formatter for the few-shot examples
Configure a formatter that will format the few-shot examples into a string. This formatter should be a `FewShotChatMessagePromptTemplate` object.

In [66]:
# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [67]:
print(few_shot_prompt.format(input=input))

Human: package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, event *MyEvent) (*MyResponse, error) {
	if event == nil {
		return nil, fmt.Errorf("received nil event")
	}
	message := fmt.Sprintf("Hello %s!", event.Name)
	return &MyResponse{Message: message}, nil
}

func main() {
	lambda.Start(HandleRequest)
}

AI: package main

import (
	"fmt"
	"log"
	"net/http"
	"os"

	"github.com/gin-gonic/gin"
)

const (
	EnvVarAzureFunctionPort = "FUNCTIONS_PORT"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx *gin.Context)  {
	if ctx.Request.Body == nil {
		errorMsg := "received nil event"
		ctx.JSON(http.StatusBadRequest, gin.H{"error": errorMsg})
		return 
	}

	var event MyEvent
	err := ctx.Bind(&event)
	if err != nil {
		errorMs

In [68]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_input),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [69]:
print(final_prompt.format(input=input))

System: You are an azure assistant that converts aws lambda code into azure functions. 
    You do it by using the following steps:
    
    1. Determine if the code corresponds to a lambda function, by checking its imports
    2. If it isn't a lambda function, return it as is
    3. If it is a lambda function then convert the file to an azure function
   
   Return the resultant code, and only code do not add any explanation.
   
   If you don't know the answer return the original code with the following comment: 
   `// Could not convert the code, please check the class`, followed by a commented explanation of what happened.
   
Human: package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, event *MyEvent) (*MyResponse, error) {
	if event == nil {
		return nil, fmt.Errorf("received nil event")
	}
	message :

### Use with an LLM
Now, you can connect your model to the few-shot prompt.

In [ ]:
conversation = ConversationChain(llm=llm)

result = conversation.run(final_prompt.format_messages(input=input))

In [ ]:
print(result)

### Check if compiles 

The idea would be to compile the classes and if it doesn't compile tell the LLM the issue and retry.

The result doesn't seem to compile let's ask the LLM again.

In [ ]:
conversation.run("the code doesn't compile")

### Let's try with format instructions

It's possible to tell our LLM how we expect to get our response back using output parsers, let's try it out

### Testing more inputs

Try again using new inputs

In [ ]:
lambda_code_1 = '''
package main

import (
	"context"
	"encoding/json"
	"github.com/aws/aws-lambda-go/events"
	runtime "github.com/aws/aws-lambda-go/lambda"
	"github.com/aws/aws-lambda-go/lambdacontext"
	"github.com/aws/aws-sdk-go/aws/session"
	"github.com/aws/aws-sdk-go/service/lambda"
	"log"
	"os"
)

var client = lambda.New(session.New())

func callLambda() (string, error) {
	input := &lambda.GetAccountSettingsInput{}
	req, resp := client.GetAccountSettingsRequest(input)
	err := req.Send()
	output, _ := json.Marshal(resp.AccountUsage)
	return string(output), err
}

func handleRequest(ctx context.Context, event events.SQSEvent) (string, error) {
	// event
	eventJson, _ := json.MarshalIndent(event, "", "  ")
	log.Printf("EVENT: %s", eventJson)
	// environment variables
	log.Printf("REGION: %s", os.Getenv("AWS_REGION"))
	log.Println("ALL ENV VARS:")
	for _, element := range os.Environ() {
		log.Println(element)
	}
	// request context
	lc, _ := lambdacontext.FromContext(ctx)
	log.Printf("REQUEST ID: %s", lc.AwsRequestID)
	// global variable
	log.Printf("FUNCTION NAME: %s", lambdacontext.FunctionName)
	// context method
	deadline, _ := ctx.Deadline()
	log.Printf("DEADLINE: %s", deadline)
	// AWS SDK call
	usage, err := callLambda()
	if err != nil {
		return "ERROR", err
	}
	return usage, nil
}

func main() {
	runtime.Start(handleRequest)
}
'''

In [ ]:
conversation = ConversationChain(llm=llm)

result = conversation.run(final_prompt.format_messages(input=lambda_code_1))

In [ ]:
print(result)